In [ ]:
from Bio import SeqIO
import pandas as pd
import numpy as np


chrdict = {}

# !!!!! MODIFICATE IL PERCORSO DEL FILE
# !!!!! MODIFICATE IL PERCORSO DEL FILE
# !!!!! MODIFICATE IL PERCORSO DEL FILE
# !!!!! MODIFICATE IL PERCORSO DEL FILE

for i, record in enumerate(SeqIO.parse("C:\Riccardo\Magistrale_ing_inf\AI_for_Bioinformatics\progetto_37\controllo_mappaggio\GRCh37.primary_assembly.genome.fa\GRCh37.primary_assembly.genome.fa", "fasta")):
    print(f"{record.id}")
    chrdict[record.id] = record.seq


# LE SE SEQUENZE DI BASI AZOTATE SONO MEMORIZZATE IN UN DIZIONARIO CHE USA COME CHIAVE "chr#" dove # indica
# il numero del cromosoma oppure X/Y/M (M non ci interessa perché é il DNA mitocondriale che non viene usato per questo caso)


# UA INIZIA INVECE LA PARTE DI LETTURA DEL FILE CONTENENTE I TSS


def extract_p(p):
    return p.split("@")[0]

def extract_gene_name(string):
    string =  string.split("@")[1]
    return string.split(",")[0]

v1 = np.vectorize(extract_p)
v2 = np.vectorize(extract_gene_name)

# ANCHE QUA CAMBIATE IL PERCORSO DEL FILE

df = pd.read_csv("C:\Riccardo\Magistrale_ing_inf\AI_for_Bioinformatics\progetto_37\costruzione_dataset\TSS_human.bed\TSS_human.bed", delimiter="\t", skiprows=1, names=["chromosome_name", "start", "end", "locus_id", "color", "strand", "start2", "end2", "RGB"])

df['p'] = v1(df['locus_id'].values)
df['gene_name'] = v2(df['locus_id'].values)


result_df = df.loc[df.groupby("gene_name")["p_value"].idxmax()]

result_df = result_df.drop(columns=["p_value"])

In [ ]:
result_df